In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import PIL as Image
import os
from random import shuffle
import glob
import tensorflow as tf
from sklearn.utils import shuffle

In [2]:
import cv2
import os
from os import listdir
from os.path import isfile, join

In [46]:
path="C:/Users/asd/Desktop/Winter Training/Classification_Assignment_Data"
cats=[]
dogs=[]
for i in range(1440):
    cats.append(path+"/cats/cat."+str(i)+'.jpg')
    dogs.append(path+"/dogs/dog."+str(i)+'.jpg')
print(np.shape(cats))
print(np.shape(dogs))
totalData=cats+dogs
print(np.shape(totalData))

(1440,)
(1440,)
(2880,)


In [47]:
batchhsize=64

In [48]:
labelDatadash=np.arange(2880)
labelDatadash[0:1440]=0
labelDatadash[1440:2880]=1
oneHotDash=np.eye(2)[labelDatadash]
labelData=np.reshape(oneHotDash,(2880,2))
print(labelData.shape)


(2880, 2)


In [49]:
def unison_shuffle(a, b):
    X,Y=shuffle(a, b, random_state=0)
    return X, Y 
#     assert len(a) == len(b)
#     p = np.random.permutation(len(a))
#     return a[p], b[p]

In [50]:
totalData,labelData=unison_shuffle(totalData,labelData)
print(np.shape(totalData))

(2880,)


In [51]:
trainData=totalData[0:2304]
trainLabel=labelData[0:2304]
valData=totalData[2304:2880]
valLabel=labelData[2304:2880]

In [52]:
print(trainData[6],trainLabel[1])

C:/Users/asd/Desktop/Winter Training/Classification_Assignment_Data/cats/cat.1129.jpg [ 0.  1.]


In [53]:
def createBatch(start,batchSize,newSize):
    batchAddr=trainData[start:start+batchSize]
    batchData=[]
    for j in range(batchSize):
        img=cv2.imread(batchAddr[j])
        img.resize(newSize,newSize,3)
        batchData.append(img)
    batchLabel=trainLabel[start:start+batchSize]
    return batchData,batchLabel

In [54]:
x = tf.placeholder(tf.float32, shape=[None,128,128,3])
y_ = tf.placeholder(tf.float32, shape=[None,2])

### Weight Initialization

In [55]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)#truncated, anything outside -0.2 and 0.2 has a probability of occurence 0
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)#constant array of 0.1 with whatever shape we give to it
    return tf.Variable(initial)

### Convolution and pooling functions

In [56]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### First Layer

In [57]:
W_conv1 = weight_variable([5, 5, 3, 32])#5X5 filter 3 are the in channel (gray-scale) and there are 32 feature maps
b_conv1 = bias_variable([32])#32 feature maps

In [58]:
#x_image = tf.reshape(x, [-1, 28, 28, 1])

In [59]:
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)#batchSizeX128X128X32. Here 64 is the batch size (assumed)
h_pool1 = max_pool_2x2(h_conv1)#max pooling batchSizeX64X64X32

### Second conv layer

In [60]:
W_conv2 = weight_variable([5, 5, 32, 64])#batchSize64
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)#batchSizeX32X32X64

In [61]:
W_fc1 = weight_variable([32 * 32 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 32*32*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [62]:
keep_prob = tf.placeholder(tf.float32)#probability of retaining the neurons. 0=all neurons dropped, 1 means no neuron is dropped
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [63]:
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [64]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [68]:
train_step = tf.train.RMSPropOptimizer(1e-2).minimize(cross_entropy)

In [69]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [70]:
with tf.Session() as sess:#declaring tf.Session as sess
    sess.run(tf.global_variables_initializer())#initializing all the variables we used
    for j in range(10):
        for i in range(24):#number of iterations we are using
            batchData,batchLabel=createBatch(i*96,96,128)
            #print(batchData.shape)
            train_accuracy = accuracy.eval(feed_dict={x: batchData, y_: batchLabel, keep_prob: 1.0})#training accuracy calculated with no dropout probabilty
            print('step %d, training accuracy %g' % (i, train_accuracy))
            train_step.run(feed_dict={x: batchData, y_: batchLabel, keep_prob: 0.7})#dropout probabilty during training is kept 0.5
    print('test accuracy %g' % accuracy.eval(feed_dict={x:valData, y_: valLabel, keep_prob: 1.0}))#test accuracy


step 0, training accuracy 0.427083
step 1, training accuracy 0.5
step 2, training accuracy 0.458333
step 3, training accuracy 0.53125
step 4, training accuracy 0.479167
step 5, training accuracy 0.520833
step 6, training accuracy 0.4375
step 7, training accuracy 0.46875
step 8, training accuracy 0.447917
step 9, training accuracy 0.447917
step 10, training accuracy 0.479167
step 11, training accuracy 0.614583
step 12, training accuracy 0.489583
step 13, training accuracy 0.510417
step 14, training accuracy 0.4375
step 15, training accuracy 0.520833
step 16, training accuracy 0.5625
step 17, training accuracy 0.46875
step 18, training accuracy 0.552083
step 19, training accuracy 0.53125
step 20, training accuracy 0.46875
step 21, training accuracy 0.520833
step 22, training accuracy 0.416667
step 23, training accuracy 0.510417
step 0, training accuracy 0.59375
step 1, training accuracy 0.489583
step 2, training accuracy 0.583333
step 3, training accuracy 0.541667
step 4, training accura

KeyboardInterrupt: 